In [ ]:
%load_ext autoreload
import numpy as np
import seaborn as sns
import os
from EQALS import ClassificationModelsimple
import geopandas as gpd
import tensorflow as tf
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
clfmdl=ClassificationModelsimple.LandslideModel()
clfmdl.getclassificationModel(in_num=8, out_num=1)
clfmdl.model.load_weights('checkpointsUSGSv2/')

In [ ]:
def trainmodel(model,xdata,ydata):
    
    NUMBER_EPOCHS = 200
    filepath='checkpointsUSGSv2'
    BATCH_SIZE=32
    
    model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor="val_auc",
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        options=None
    )
    print(type(xdata),type(ydata))
    hist = model.fit(x=xdata,
                     y=ydata,
                     epochs=NUMBER_EPOCHS,
                     batch_size=BATCH_SIZE,
                    #  validation_split=0.2,#auto validate using 20% of random samples at each epoch
                     verbose=1,class_weight = {0: 1, 1: 5}

                    )
    return hist


In [ ]:
#Spatial cross validation
#load data
df=gpd.read_file('Data/USGS/NepalEqUSGSSpval.shp')
constcols=['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGV_Usgs']
Xdata=df[constcols].to_numpy()
Ydata=df.Landslide.to_numpy()
max=Xdata.max(axis=0)
min=Xdata.min(axis=0)
norm_xdata=(Xdata-min)/(max-min)
def spatialCrossVal():
    for i in range(1,11):
        print(i)
        clfmdl=ClassificationModelsimple.LandslideModel()
        clfmdl.getclassificationModel(in_num=8, out_num=1)
        clfmdl.getOptimizer()
        clfmdl.compileModel()
        train_index=df.index[df['id']!=i].tolist()
        test_index=df.index[df['id']==i].tolist()
        print(f"Number of train set{len(train_index)}")
        X_train=norm_xdata[train_index]
        y_train=Ydata[train_index]
        X_test=norm_xdata[test_index]
        y_test=Ydata[test_index]
        trainmodel(clfmdl.model,np.array(X_train,dtype=np.float32),np.expand_dims(np.array(y_train,dtype=np.float32),axis=-1))
        preds=clfmdl.model.predict(X_test)
        np.save(f'crossval_results/SpPredsv2{str(i)}.npy',preds)
        np.save(f'crossval_results/SpTruthsv2{str(i)}.npy',y_test)
        del clfmdl
        i+=1

In [ ]:
spatialCrossVal()

In [ ]:
from sklearn.model_selection import KFold
df=gpd.read_file('Data/USGS/NepalEqUSGS.shp')
constcols=['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGV_Usgs']
Xdata=df[constcols].to_numpy()
Ydata=df.Landslide.to_numpy()
max=Xdata.max(axis=0)
min=Xdata.min(axis=0)
norm_xdata=(Xdata-min)/(max-min)
rcl_indexes=[]
def randomCrossVal():
    kf = KFold(n_splits=10,random_state=42,shuffle=True)
    kf.get_n_splits(norm_xdata)
    i=0
    for train_index, test_index in kf.split(norm_xdata):
        # rcl_indexes.append(test_index)
        print(i)
        clfmdl=ClassificationModelsimple.LandslideModel()
        clfmdl.getclassificationModel(in_num=8, out_num=1)
        clfmdl.getOptimizer()
        clfmdl.compileModel()
        X_train=norm_xdata[train_index]
        y_train=Ydata[train_index]
        X_test=norm_xdata[test_index]
        y_test=Ydata[test_index]
        trainmodel(clfmdl.model,np.array(norm_xdata,dtype=np.float32),np.expand_dims(np.array(Ydata,dtype=np.float32),axis=-1))
        # preds=clfmdl.model.predict(X_test)
        np.save(f'crossval_results_Truths{str(i)}.npy',y_test)
        preds=clfmdl.model.predict(norm_xdata)
        break
        i+=1
        
    return Ydata,preds
        # del clfmdl
        



In [ ]:
Ydata,preds=randomCrossVal()

In [ ]:
#predict and save
#prepare data
import shap
import tensorflow as tf
import geopandas as gpd
import numpy as np
df=gpd.read_file('Data/AllDataV2.gpkg')
constcols=['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','pgvhz_f_0_5']
Xdata=df[constcols].to_numpy()
Ydata=df.Landslide.to_numpy()
max=Xdata.max(axis=0)
min=Xdata.min(axis=0)
norm_xdata=(Xdata-min)/(max-min)
model=tf.keras.models.load_model('checkpoints')
preds=model.predict(norm_xdata)

In [ ]:
sklearn.metrics.auc(fpr,tpr)

In [ ]:
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
fpr,tpr,thresholds=sklearn.metrics.roc_curve(Ydata, preds)
from matplotlib.pyplot import figure

figure(figsize=(4, 4), dpi=500)

lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % 0.8577,
)
ax=plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Landslide Classification")
plt.text(0.38, 0.11,"Accuracy=0.8131")
plt.legend(loc="lower right")
plt.axis('square')

plt.savefig('Exaiplots/roc.pdf')
plt.show()


In [ ]:
#plot 
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np 
figure(figsize=(4, 4), dpi=100)
Aucs=[]
for i in range(1,11):
    preds=np.load(f'crossval_results/SpPredsv2{str(i)}.npy')
    truths=np.load(f'crossval_results/SpTruthsv2{str(i)}.npy')
    fpr,tpr,thresholds=sklearn.metrics.roc_curve(truths.flatten(), preds.flatten())
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=0.5,
        label=f"ValArea{str(i)}",
    )
    Aucs.append(sklearn.metrics.auc(fpr,tpr))

ax=plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Spatial Crossvalidation")
# plt.text(0.38, 0.11,"Accuracy=0.8131")
# plt.legend(loc="lower right")
plt.axis('square')
plt.tight_layout()
plt.savefig('Exaiplots/rocSpVal.pdf')
plt.show()


In [ ]:
#plot 
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np 
figure(figsize=(4, 4), dpi=100)
Aucs2=[]
for i in range(0,10):
    preds=np.load(f'crossval_results/crossval_results_Preds{str(i)}.npy')
    truths=np.load(f'crossval_results/crossval_results_Truths{str(i)}.npy')
    fpr,tpr,thresholds=sklearn.metrics.roc_curve(truths.flatten(), preds.flatten())
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=0.5,
        label=f"RandomArea{str(i+1)}",
    )
    Aucs2.append(sklearn.metrics.auc(fpr,tpr))

ax=plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Random Crossvalidation")
# plt.text(0.38, 0.11,"Accuracy=0.8131")
# plt.legend(loc="lower right")
plt.axis('square')
plt.tight_layout()
plt.savefig('Exaiplots/rocrandVal.pdf')
plt.show()


In [ ]:
sns.set_theme()
sns.boxplot(Aucs2,color='azure')
plt.xlim(0.5,1)
plt.savefig('Exaiplots/violin_randomval.pdf')


In [ ]:
sns.boxplot(Aucs,color='aliceblue')
plt.xlim(0.5,1)
plt.savefig('Exaiplots/violin_spval.pdf')

In [ ]:
# plot all confusion maps


In [ ]:
#plot 
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np 
df=gpd.read_file('Data/USGS/NepalEqUSGS.shp')
for i in range(0,10):
    print(i)
    preds=np.load(f'crossval_results/crossval_results_Preds{str(i)}.npy')
    truths=np.load(f'crossval_results/crossval_results_Truths{str(i)}.npy')

    #confusion  map
    confusiondata=np.empty(truths.shape,dtype=object)
    confusiondata[np.bitwise_and(truths.flatten()==1,np.rint(preds.flatten())==1)]='False Positive'
    confusiondata[np.bitwise_and(truths.flatten()==0,np.rint(preds.flatten())==1)]='True Positive'
    confusiondata[np.bitwise_and(truths.flatten()==1,np.rint(preds.flatten())==0)]='False Negative'
    confusiondata[np.bitwise_and(truths.flatten()==0,np.rint(preds.flatten())==0)]='True Negative'
    sel_idx=rcl_indexes[i].tolist()
    df.loc[sel_idx,'rcl_confusion']=confusiondata
df_rc=df

In [ ]:
df=gpd.read_file('Data/USGS/NepalEqUSGSSpval.shp')
for i in range(1,11):
    preds=np.load(f'crossval_results/SpPredsv2{str(i)}.npy')
    truths=np.load(f'crossval_results/SpTruthsv2{str(i)}.npy')
    print(i)
    #confusion  map
    confusiondata=np.empty(truths.shape,dtype=object)
    confusiondata[np.bitwise_and(truths.flatten()==1,np.rint(preds.flatten())==1)]='False Positive'
    confusiondata[np.bitwise_and(truths.flatten()==0,np.rint(preds.flatten())==1)]='True Positive'
    confusiondata[np.bitwise_and(truths.flatten()==1,np.rint(preds.flatten())==0)]='False Negative'
    confusiondata[np.bitwise_and(truths.flatten()==0,np.rint(preds.flatten())==0)]='True Negative'
    sel_idx=df.index[df['id']==i].tolist()
    df.loc[sel_idx,'scl_confusion']=confusiondata
df_sc=df

In [ ]:
import contextily as cx
df_wm = df_sc.to_crs(epsg=3857)
ax=df_wm.plot(column='scl_confusion',legend=False,figsize=(10, 10), alpha=0.7)
cx.add_basemap(ax,source=cx.providers.Esri.WorldShadedRelief)

In [ ]:
ax.get_figure().savefig('Exaiplots/confusionmap_scv.pdf',dpi=500,facecolor=ax.get_facecolor())

In [ ]:
import contextily as cx
df_wm = df.to_crs(epsg=3857)
ax=df_wm.plot(column='confusion',legend=True,figsize=(10, 10), alpha=0.7)
cx.add_basemap(ax,source=cx.providers.Esri.WorldShadedRelief)

In [ ]:
ax.get_figure().savefig('Exaiplots/confusionmap_rcv.pdf',dpi=500,facecolor=ax.get_facecolor())

In [ ]:
ax=df_rc.rcl_confusion.value_counts().plot(kind='barh',logx=True,xlim=(10,1e4))
ax.get_figure().savefig('Exaiplots/barplot_rcv.pdf',dpi=500,facecolor=ax.get_facecolor())

In [ ]:
ax=df_sc.scl_confusion.value_counts().plot(kind='barh',logx=True,xlim=(10,1e4))
ax.get_figure().savefig('Exaiplots/barplot_scv.pdf',dpi=500,facecolor=ax.get_facecolor())

In [ ]:
ax=df.confusion.value_counts().plot(kind='barh',logx=True,xlim=(10,1e4))
ax.get_figure().savefig('Exaiplots/barplot_best.pdf',dpi=500,facecolor=ax.get_facecolor())